In [1]:
%matplotlib inline

In [2]:
# Test:
# Max length:
# 10
# 40
# Drop-out: 
# 0.4
# 0.7
# LSTM---> best performance
# RNN
# Compare performance between Decoder and AttnDecoder


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import sys
path_to_final_project = "/content/drive/Shareddrives/Deep Learning Final Project/"
sys.path.append(path_to_final_project)
checkpoint_folder_path = "checkpoint/Reza/fake reply generation/max length 40/"
encoder_checkpoint_name = "encoder-best.pth"
decoder_checkpoint_name = "decoder-best.pth"


NLP From Scratch: Translation with a Sequence to Sequence Network and Attention
*******************************************************************************
**Author**: `Sean Robertson <https://github.com/spro/practical-pytorch>`_

This is the third and final tutorial on doing "NLP From Scratch", where we
write our own classes and functions to preprocess the data to do our NLP
modeling tasks. We hope after you complete this tutorial that you'll proceed to
learn how `torchtext` can handle much of this preprocessing for you in the
three tutorials immediately following this one.

In this project we will be teaching a neural network to translate from
French to English.

::

    [KEY: > input, = target, < output]

    > il est en train de peindre un tableau .
    = he is painting a picture .
    < he is painting a picture .

    > pourquoi ne pas essayer ce vin delicieux ?
    = why not try that delicious wine ?
    < why not try that delicious wine ?

    > elle n est pas poete mais romanciere .
    = she is not a poet but a novelist .
    < she not not a poet but a novelist .

    > vous etes trop maigre .
    = you re too skinny .
    < you re all alone .

... to varying degrees of success.

This is made possible by the simple but powerful idea of the `sequence
to sequence network <https://arxiv.org/abs/1409.3215>`__, in which two
recurrent neural networks work together to transform one sequence to
another. An encoder network condenses an input sequence into a vector,
and a decoder network unfolds that vector into a new sequence.

.. figure:: /_static/img/seq-seq-images/seq2seq.png
   :alt:

To improve upon this model we'll use an `attention
mechanism <https://arxiv.org/abs/1409.0473>`__, which lets the decoder
learn to focus over a specific range of the input sequence.

**Recommended Reading:**

I assume you have at least installed PyTorch, know Python, and
understand Tensors:

-  https://pytorch.org/ For installation instructions
-  :doc:`/beginner/deep_learning_60min_blitz` to get started with PyTorch in general
-  :doc:`/beginner/pytorch_with_examples` for a wide and deep overview
-  :doc:`/beginner/former_torchies_tutorial` if you are former Lua Torch user


It would also be useful to know about Sequence to Sequence networks and
how they work:

-  `Learning Phrase Representations using RNN Encoder-Decoder for
   Statistical Machine Translation <https://arxiv.org/abs/1406.1078>`__
-  `Sequence to Sequence Learning with Neural
   Networks <https://arxiv.org/abs/1409.3215>`__
-  `Neural Machine Translation by Jointly Learning to Align and
   Translate <https://arxiv.org/abs/1409.0473>`__
-  `A Neural Conversational Model <https://arxiv.org/abs/1506.05869>`__

You will also find the previous tutorials on
:doc:`/intermediate/char_rnn_classification_tutorial`
and :doc:`/intermediate/char_rnn_generation_tutorial`
helpful as those concepts are very similar to the Encoder and Decoder
models, respectively.

And for more, read the papers that introduced these topics:

-  `Learning Phrase Representations using RNN Encoder-Decoder for
   Statistical Machine Translation <https://arxiv.org/abs/1406.1078>`__
-  `Sequence to Sequence Learning with Neural
   Networks <https://arxiv.org/abs/1409.3215>`__
-  `Neural Machine Translation by Jointly Learning to Align and
   Translate <https://arxiv.org/abs/1409.0473>`__
-  `A Neural Conversational Model <https://arxiv.org/abs/1506.05869>`__


**Requirements**



In [5]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Loading data files
==================

The data for this project is a set of many thousands of English to
French translation pairs.

`This question on Open Data Stack
Exchange <https://opendata.stackexchange.com/questions/3888/dataset-of-sentences-translated-into-many-languages>`__
pointed me to the open translation site https://tatoeba.org/ which has
downloads available at https://tatoeba.org/eng/downloads - and better
yet, someone did the extra work of splitting language pairs into
individual text files here: https://www.manythings.org/anki/

The English to French pairs are too big to include in the repo, so
download to ``data/eng-fra.txt`` before continuing. The file is a tab
separated list of translation pairs:

::

    I am cold.    J'ai froid.

.. Note::
   Download the data from
   `here <https://download.pytorch.org/tutorial/data.zip>`_
   and extract it to the current directory.



Similar to the character encoding used in the character-level RNN
tutorials, we will be representing each word in a language as a one-hot
vector, or giant vector of zeros except for a single one (at the index
of the word). Compared to the dozens of characters that might exist in a
language, there are many many more words, so the encoding vector is much
larger. We will however cheat a bit and trim the data to only use a few
thousand words per language.

.. figure:: /_static/img/seq-seq-images/word-encoding.png
   :alt:





We'll need a unique index per word to use as the inputs and targets of
the networks later. To keep track of all this we will use a helper class
called ``Lang`` which has word → index (``word2index``) and index → word
(``index2word``) dictionaries, as well as a count of each word
``word2count`` to use to later replace rare words.




In [6]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

Since there are a *lot* of example sentences and we want to train
something quickly, we'll trim the data set to only relatively short and
simple sentences. Here the maximum length is 10 words (that includes
ending punctuation) and we're filtering to sentences that translate to
the form "I am" or "He is" etc. (accounting for apostrophes replaced
earlier).




In [7]:
MAX_LENGTH = 40

def filterPair(p):
  return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

The full process for preparing the data is:

-  Read text file and split into lines, split lines into pairs
-  Normalize text, filter by length and content
-  Make word lists from sentences in pairs




In [8]:
def sentences_preprocessing(panda_dataset, header_com, header_reply):

  # all this would do is remove rows if elements were missing in any of the columns. 
  panda_dataset = panda_dataset.dropna()

  #remove \r (this is where you are running into vocabulary problems.)
  panda_dataset = panda_dataset.replace(to_replace=r'\r', value='', regex=True)

  #remove \n (this is where you are running into vocabulary problems.)
  panda_dataset = panda_dataset.replace(to_replace='\n', value='', regex=True)

  #convert pandas to list
  list_dataset = np.transpose([panda_dataset[header_com],panda_dataset[header_reply]]).tolist()
  return list_dataset, panda_dataset

In [9]:
import pandas as pd
import numpy as np

TRAINING_CSV = 'Childish Gambino - This Is America (Official Video)_refined.csv'
VALIDATION_CSV = 'Childish Gambino - This Is America (Official Video)_validation_comments_top_n.csv'

In [10]:
training_raw_data = pd.read_csv(path_to_final_project+'data/'+TRAINING_CSV, error_bad_lines=False, encoding='utf-8-sig')
training_raw_data.head()

,no_space_comment,no_space_reply,Replies,Likes,first_2_keywords_in_no_space_comment,first_2_words_in_no_space_comment
0,"donald glover jack of all trades , master of a...",https //youtu . be/rqn-dsmlf6er,25,6378,donald glover,donald glover
1,no one \r \r \r \r \r \r\r\r\n \r \r \r \r \r ...,"extraterrestrial it's a joke , shut yo sensiti...",394,75231,one,no one
2,me brings bulletproof shield to school \r \r \...,lol,273,53125,brings bulletproof,me brings
3,2020 someone ? very niceee ! !,yes 2020 trash,60,2195,2020 someone,2020 someone
4,"childish gambino 'shoots someone' "" this is am...",oop- face_with_hand_over_mouth flushed_face ra...,211,41154,childish gambino,childish gambino


In [11]:
test_raw_data = pd.read_csv(path_to_final_project+'data/'+VALIDATION_CSV, error_bad_lines=False, encoding='utf-8-sig')
test_raw_data.head()

,Validation Comment,Validation Reply,first_2_keywords_in_Validation Comment,first_2_words_in_Validation Comment
0,"at the current state , we can do anything in a...",no reply,current state,at the
1,i miss donald as troy in community when he lef...,no reply,miss donald,i miss
2,"when he said "" this is america "" i felt that",no reply,said america,when he
3,"love him or hate him , hes telling the truth",no reply,love hate,love him
4,"when the church people were singing i heard "" ...",no reply,church people,when the


In [12]:
training_data, df_training = sentences_preprocessing(training_raw_data, 'no_space_comment', 'no_space_reply')
print(np.shape(training_data))
df_training.head()

(5736, 2)


,no_space_comment,no_space_reply,Replies,Likes,first_2_keywords_in_no_space_comment,first_2_words_in_no_space_comment
0,"donald glover jack of all trades , master of a...",https //youtu . be/rqn-dsmlf6er,25,6378,donald glover,donald glover
1,no one childish gambinos spine rig...,"extraterrestrial it's a joke , shut yo sensiti...",394,75231,one,no one
2,me brings bulletproof shield to school t...,lol,273,53125,brings bulletproof,me brings
3,2020 someone ? very niceee ! !,yes 2020 trash,60,2195,2020 someone,2020 someone
4,"childish gambino 'shoots someone' "" this is am...",oop- face_with_hand_over_mouth flushed_face ra...,211,41154,childish gambino,childish gambino


In [13]:
test_data, df_test = sentences_preprocessing(test_raw_data, 'Validation Comment', 'Validation Reply')
print(np.shape(test_data))
df_test.head()

(340, 2)


,Validation Comment,Validation Reply,first_2_keywords_in_Validation Comment,first_2_words_in_Validation Comment
0,"at the current state , we can do anything in a...",no reply,current state,at the
1,i miss donald as troy in community when he lef...,no reply,miss donald,i miss
2,"when he said "" this is america "" i felt that",no reply,said america,when he
3,"love him or hate him , hes telling the truth",no reply,love hate,love him
4,"when the church people were singing i heard "" ...",no reply,church people,when the


In [14]:
  # #add space between words and some character
  # panda_dataset = panda_dataset.replace(to_replace=r':', value=':', regex=True) 
  # panda_dataset = panda_dataset.replace(to_replace=r',', value=' ,', regex=True) 
  # panda_dataset = panda_dataset.replace(to_replace=r'\'', value=' \'', regex=True)
  # panda_dataset = panda_dataset.replace(to_replace=r'\`', value=' \`', regex=True)
  # panda_dataset = panda_dataset.replace(to_replace=r'\.', value=' .', regex=True)
  # panda_dataset = panda_dataset.replace(to_replace=r'\(', value='( ', regex=True)
  # panda_dataset = panda_dataset.replace(to_replace=r'\)', value=' )', regex=True)
  # panda_dataset = panda_dataset.replace(to_replace=r'\[', value='( ', regex=True)
  # panda_dataset = panda_dataset.replace(to_replace=r'\]', value=' )', regex=True)
  # panda_dataset = panda_dataset.replace(to_replace=r'\{', value='( ', regex=True)
  # panda_dataset = panda_dataset.replace(to_replace=r'\}', value=' )', regex=True)
  # panda_dataset = panda_dataset.replace(to_replace=r'\?', value=' ?', regex=True)
  # panda_dataset = panda_dataset.replace(to_replace=r'\!', value=' !', regex=True)
  # #remove asteriks
  # panda_dataset = panda_dataset.replace(to_replace=r'\*', value='', regex=True)
  # panda_dataset = panda_dataset.replace(to_replace='\n', value=' \n ', regex=True)
  # #remove some repetitive spaces 
  # space = ' '
  # for i in range(10):
  #   space +=' '
  #   panda_dataset = panda_dataset.replace(to_replace=space, value=' ', regex=True) #add space after \n


In [15]:
# This gets me the data trimmed so that I can create the vocabulary comparison
def prepare_data_vocab(comment_reply, reverse=False):
    pairs = filterPairs(comment_reply) 
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    input_sentence = [i[0] for i in pairs]
    output_sentence = [i[1] for i in pairs]

    return input_sentence, output_sentence

In [16]:
print("########### Training Set ###########")
input_sentence, output_sentence = prepare_data_vocab(training_data, True)

print("\n","########### Test Set ###########")
test_input_sentence, test_output_sentence = prepare_data_vocab(test_data, True)


########### Training Set ###########
Trimmed to 4681 sentence pairs
Counting words...

 ########### Test Set ###########
Trimmed to 339 sentence pairs
Counting words...


In [17]:
import itertools
def unique(list1): 
      
    # insert the list to the set 
    list_set = set(list1) 
    # convert the set to the list 
    unique_list = (list(list_set))
    
    return unique_list

def top_n_words(input_sentence):

    words = []
    for comment in input_sentence:
        temp = comment.split()
        words.append(temp)
                     
    unique_word_list = list(itertools.chain(*words))
    
    unique_word_list = unique(unique_word_list)
    
    return unique_word_list

In [18]:
print("########### Training Set ###########")
unique_word_list_train = top_n_words(input_sentence)
print(len(unique_word_list_train))
print(unique_word_list_train)
print("\n","########### Test Set ###########")
unique_word_list_test = top_n_words(test_input_sentence)
print(len(unique_word_list_test))
print(unique_word_list_test)

########### Training Set ###########
5222
['ak', 'republican', '99', 'relations', 'uwu', 'represent', 'imagined', 'face_with_rolling_eyes', 'honestly', 'without', 'rlly', 'hurts', 'brutally', 'sighs', '200%', 'repetitive', '?', 'belongs', 'escape', 'passing', "someone'", 'dropped', '@donald', 'haters', 'karen', 'injected', 'minutes', 'aloud', 'eyed', 'glad', 'europeans', 'bibliotheca', 'multiple', 'cruel', 'suddenly', 'revisit', 'sane', 'police_officer', 'hand_with_fingers_splayed', 'disagree', 'poorer', 'crime', 'robbed', 'spider', 'goin', 'chocolate', 'suxxx', 'abdomen', "shouldn't", '52-1', '000', '735k', 'greater', 'disaster', 'describing', 'ia', 'waving_hand', 'lowkey', 'hows', 'ctm', 'recomended', 'verbally', 'unfunny', 'species', 'person', 'super', 'blocks', 'far', 'christ', 'arbour', 'tho', 'militarys', 'singin', 'defend', 'idk', 'tourist', 'peacefully', 'estate', 'insane', 'plez', 'guns', '57', 'realise', 'realy', 'weapon', 'infowars', 'psychopath', 'thay', 'educate', 'faces',

In [19]:
import csv
csv_vocab_validation = 'vocabulary_validation.csv'
save_csv_location = path_to_final_project+'data/'+csv_vocab_validation
val_comment_vocab_check = []
with open(save_csv_location, 'w', newline='', encoding='utf-8-sig') as csvfile:
    fieldnames = ['Validation Comment', 'Validation Reply']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    
    for sentence in test_input_sentence:
        individual_words = sentence.split()
        # print(individual_words)
        if all(item in unique_word_list_train for item in individual_words):
            val_comment_vocab_check.append(sentence)
            writer.writerow({'Validation Comment': sentence, 'Validation Reply': 'no reply'})
        else:
            continue
        
# print(val_comment_vocab_check)
print('here')

here


In [20]:
test_raw_data = pd.read_csv(save_csv_location, error_bad_lines=False, encoding='utf-8-sig') #----> change "data.csv" when the test set is ready
test_raw_data.head()

,Validation Comment,Validation Reply
0,"at the current state , we can do anything in a...",no reply
1,i miss donald as troy in community when he lef...,no reply
2,"when he said "" this is america "" i felt that",no reply
3,"love him or hate him , hes telling the truth",no reply
4,"when the church people were singing i heard "" ...",no reply


In [21]:
test_data, df_test = sentences_preprocessing(test_raw_data, 'Validation Comment', 'Validation Reply')
print(np.shape(test_data))
df_test.head()

(275, 2)


,Validation Comment,Validation Reply
0,"at the current state , we can do anything in a...",no reply
1,i miss donald as troy in community when he lef...,no reply
2,"when he said "" this is america "" i felt that",no reply
3,"love him or hate him , hes telling the truth",no reply
4,"when the church people were singing i heard "" ...",no reply


In [22]:
def prepare_Data(comment_reply, reverse=False):
    pairs = filterPairs(comment_reply) 
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    input_sentence = Lang("comment")
    
    output_sentence = Lang("reply")

    for pair in pairs:
        # print(count)
        input_sentence.addSentence(pair[0])
        output_sentence.addSentence(pair[1])

    print("Counted words:")
    print(input_sentence.name, input_sentence.n_words)
    print(output_sentence.name, output_sentence.n_words)
    return input_sentence, output_sentence, pairs

    return input_sentence, output_sentence, pairs

print("########### Training Set ###########")
input_sentence, output_sentence, pairs = prepare_Data(training_data, True)
print(random.choice(pairs))
print("\n","########### Test Set ###########")
test_input_sentence, test_output_sentence, test_pairs = prepare_Data(test_data, True)
print(random.choice(test_pairs))

########### Training Set ###########
Trimmed to 4681 sentence pairs
Counting words...
Counted words:
comment 5225
reply 4968
['me brings bulletproof shield to school       teacher why       me this is america', 'lol']

 ########### Test Set ###########
Trimmed to 275 sentence pairs
Counting words...
Counted words:
comment 711
reply 4
['i like this is america', 'no reply']


The Seq2Seq Model
=================

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A `Sequence to Sequence network <https://arxiv.org/abs/1409.3215>`__, or
seq2seq network, or `Encoder Decoder
network <https://arxiv.org/pdf/1406.1078v3.pdf>`__, is a model
consisting of two RNNs called the encoder and decoder. The encoder reads
an input sequence and outputs a single vector, and the decoder reads
that vector to produce an output sequence.

.. figure:: /_static/img/seq-seq-images/seq2seq.png
   :alt:

Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for translation between two
languages.

Consider the sentence "Je ne suis pas le chat noir" → "I am not the
black cat". Most of the words in the input sentence have a direct
translation in the output sentence, but are in slightly different
orders, e.g. "chat noir" and "black cat". Because of the "ne/pas"
construction there is also one more word in the input sentence. It would
be difficult to produce a correct translation directly from the sequence
of input words.

With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the "meaning" of the input sequence into a single
vector — a single point in some N dimensional space of sentences.




The Encoder
-----------

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.

.. figure:: /_static/img/seq-seq-images/encoder-network.png
   :alt:





In [23]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        # self.gru = nn.GRU(hidden_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        # self.rnn = nn.RNN(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        # output, hidden = self.gru(output, hidden)
        output, (hidden,_) = self.lstm(output, (hidden,hidden))
        # output, hidden = self.rnn(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder
-----------

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.




Simple Decoder
^^^^^^^^^^^^^^

In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string ``<SOS>``
token, and the first hidden state is the context vector (the encoder's
last hidden state).

.. figure:: /_static/img/seq-seq-images/decoder-network.png
   :alt:





In [24]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        # self.gru = nn.GRU(hidden_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        # self.rnn = nn.RNN(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        # output, hidden = self.gru(output, hidden)
        output, (hidden,_) = self.lstm(output, (hidden,hidden))
        # output, hidden = self.rnn(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

I encourage you to train and observe the results of this model, but to
save space we'll be going straight for the gold and introducing the
Attention Mechanism.




Attention Decoder
^^^^^^^^^^^^^^^^^

If only the context vector is passed betweeen the encoder and decoder,
that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to "focus" on a different part of
the encoder's outputs for every step of the decoder's own outputs. First
we calculate a set of *attention weights*. These will be multiplied by
the encoder output vectors to create a weighted combination. The result
(called ``attn_applied`` in the code) should contain information about
that specific part of the input sequence, and thus help the decoder
choose the right output words.

.. figure:: https://i.imgur.com/1152PYf.png
   :alt:

Calculating the attention weights is done with another feed-forward
layer ``attn``, using the decoder's input and hidden state as inputs.
Because there are sentences of all sizes in the training data, to
actually create and train this layer we have to choose a maximum
sentence length (input length, for encoder outputs) that it can apply
to. Sentences of the maximum length will use all the attention weights,
while shorter sentences will only use the first few.

.. figure:: /_static/img/seq-seq-images/attention-decoder-network.png
   :alt:





In [25]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        # self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size)
        # self.rnn = nn.RNN(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        # output, hidden = self.gru(output, hidden)
        output, (hidden,_) = self.lstm(output, (hidden,hidden))
        # output, hidden = self.rnn(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

<div class="alert alert-info"><h4>Note</h4><p>There are other forms of attention that work around the length
  limitation by using a relative position approach. Read about "local
  attention" in `Effective Approaches to Attention-based Neural Machine
  Translation <https://arxiv.org/abs/1508.04025>`__.</p></div>

Training
========

Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.




In [26]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_sentence, pair[0])
    target_tensor = tensorFromSentence(output_sentence, pair[1])
    return (input_tensor, target_tensor)

Training the Model
------------------

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.

"Teacher forcing" is the concept of using the real target outputs as
each next input, instead of using the decoder's guess as the next input.
Using teacher forcing causes it to converge faster but `when the trained
network is exploited, it may exhibit
instability <http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.378.4095&rep=rep1&type=pdf>`__.

You can observe outputs of teacher-forced networks that read with
coherent grammar but wander far from the correct translation -
intuitively it has learned to represent the output grammar and can "pick
up" the meaning once the teacher tells it the first few words, but it
has not properly learned how to create the sentence from the translation
in the first place.

Because of the freedom PyTorch's autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
``teacher_forcing_ratio`` up to use more of it.




In [27]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            # decoder_output, decoder_hidden = decoder(
            #     decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            # decoder_output, decoder_hidden = decoder(
            #     decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.




In [28]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [29]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    best_loss = 1e9

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()
    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        #saving the models
        if loss <best_loss:
          best_loss = loss
          torch.save(encoder.state_dict(), path_to_final_project + checkpoint_folder_path + encoder_checkpoint_name)
          torch.save(decoder.state_dict(), path_to_final_project + checkpoint_folder_path + decoder_checkpoint_name)

        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
            evaluateRandomly(encoder, decoder,pairs,n=2)


        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

Plotting results
----------------

Plotting is done with matplotlib, using the array of loss values
``plot_losses`` saved while training.




In [30]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.




In [31]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_sentence, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            # decoder_output, decoder_hidden = decoder(
            #     decoder_input, decoder_hidden)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_sentence.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [32]:
def evaluateRandomly(encoder, decoder, pairs, seed=5, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

Training and Evaluating
=======================

With all these helper functions in place (it looks like extra work, but
it makes it easier to run multiple experiments) we can actually
initialize a network and start training.

Remember that the input sentences were heavily filtered. For this small
dataset we can use relatively small networks of 256 hidden nodes and a
single GRU layer. After about 40 minutes on a MacBook CPU we'll get some
reasonable results.

.. Note::
   If you run this notebook you can train, interrupt the kernel,
   evaluate, and continue training later. Comment out the lines where the
   encoder and decoder are initialized and run ``trainIters`` again.




In [33]:
#--------> IMPORTATNT NOTE: running this block will re-write the checkpoints. Make sure to get a backup before uncommenting and training

hidden_size = 256
encoder1 = EncoderRNN(input_sentence.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_sentence.n_words, dropout_p=0.1).to(device)
# attn_decoder1 = DecoderRNN(hidden_size, output_sentence.n_words).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=500)

0m 19s (- 49m 24s) (500 0%) 4.6655
> whos here during the riots around the country because of george floyds death
= me , and this is the first time i've ever listened to the song .
< <EOS>

> is it just me or does childish gambino looks like donald glover ?
= lol , it is .
< <EOS>

0m 32s (- 39m 34s) (1000 1%) 4.4034
> sadly , this is america
= no its just black people in america .
< no <EOS>

> this is south africa unfortunately
= ? ?
< no <EOS>

0m 44s (- 36m 37s) (1500 2%) 4.1623
> police be trippin now that hits different now
= now ? ? this shit has been going on since 2012
< i <EOS>

> has anyone wonder if the guy who got shot in this was real and donald glover was like ''who wants to risk their life "
= yes they actually died
< i <EOS>

0m 57s (- 34m 45s) (2000 2%) 3.9654
> red_heart_selector folded_hands medium_light_skin_tone raised_fist medium_light_skin_tone raised_fist dark_skin_tone raised_fist medium_dark_skin_tone raised_fist raised_fist medium_skin_tone folded_hands medi

In [34]:
hidden_size = 256
best_encoder = EncoderRNN(input_sentence.n_words, hidden_size).to(device)
best_decoder =  AttnDecoderRNN(hidden_size, output_sentence.n_words, dropout_p=0.1).to(device)
# best_decoder =  DecoderRNN(hidden_size, output_sentence.n_words).to(device)
best_encoder.load_state_dict(torch.load(path_to_final_project + checkpoint_folder_path + encoder_checkpoint_name, map_location=torch.device('cpu')))
best_decoder.load_state_dict(torch.load(path_to_final_project + checkpoint_folder_path + decoder_checkpoint_name, map_location=torch.device('cpu')))
best_encoder.eval()
best_decoder.eval()

AttnDecoderRNN(
  (embedding): Embedding(4968, 256)
  (attn): Linear(in_features=512, out_features=40, bias=True)
  (attn_combine): Linear(in_features=512, out_features=256, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (lstm): LSTM(256, 256)
  (out): Linear(in_features=256, out_features=4968, bias=True)
)

In [35]:
#evaluate based on training set
evaluateRandomly(best_encoder, best_decoder,pairs,n=100)

> people are saying " he predicted 2020 " no he didn't this is has happening for hundreds of years and still hasn't fixed itself , no justice no peace .
= f a c t s
< i <EOS>

> this is america pouting_face 
= you knew it
< no <EOS>

> fun fact donald glover is actually childish gambino o
= oh wow , nice fact , totally didnt know
< you are you here lol <EOS>

> keep educating yourselves to hate your country . . .
= huh
< lol <EOS>

> i hope this is the kind of the stuff they show in school in the future
= game hive amen pleasssssse we need this every where
< no he didn't . <EOS>

> legit is a cry for help and im so sad please portect black artist
= i saw him in the video now wakanda fovevr
< i think the people , just because of the people , just because of the people , just because of the people , just because of the people , just because of the people , just because of the people

> im muslim because islam black =white ritch = poor       you should investigate , you will not regret
= 

In [36]:
#evaluate based on test set
evaluateRandomly(best_encoder, best_decoder,test_pairs,n=100)
# print(test_pairs)

> i long for the day this gets a billion views
= no reply
< no <EOS>

> does he pick that j back up or nah ?
= no reply
< no <EOS>

> i was watching flyingkitty and youtube recommended this to me . i mean hold up .
= no reply
< no <EOS>

> what in the hell did i just watch ? !
= no reply
< i think the people who really think he didn't . <EOS>

> this is american dream or nightmare ? lol
= no reply
< yes <EOS>

> does anyone else still come here every time they hear about a mass shooting that happened in the united states ?
= no reply
< yes <EOS>

> blows my mind that this was 2 whole years ago
= no reply
< you you mean the fuck he didn't . <EOS>

> let's go back to 2018
= no reply
< i think the people , just because of the people , who really think he didn't . <EOS>

> do not be embarrassed why people like it to this day , because america and in 2020 america
= no reply
< yes , <EOS>

> i came here after the this is america , so call me maybe
= no reply
< yeah i know <EOS>

> just being

In [37]:
#Run below two lines (in google colab), installation steps to get .pdf of the notebook

!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pandoc is already the newest version (1.19.2.4~dfsg-1build4).
texlive is already the newest version (2017.20180305-1).
texlive-latex-extra is already the newest version (2017.20180305-2).
texlive-xetex is already the newest version (2017.20180305-1).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [38]:
import subprocess
import time
# Wait for 5 minuts so that colab saves the notebook
time.sleep(5*60)
!jupyter nbconvert --to HTML "/content/drive/Shareddrives/Deep Learning Final Project/fake_reply_generation.ipynb"
!jupyter nbconvert --to PDF "/content/drive/Shareddrives/Deep Learning Final Project/fake_reply_generation.ipynb"
ipynb_address = path_to_final_project + checkpoint_folder_path
subprocess.call(["mv", "/content/drive/Shareddrives/Deep Learning Final Project/fake_reply_generation.html", ipynb_address])
subprocess.call(["mv", "/content/drive/Shareddrives/Deep Learning Final Project/fake_reply_generation.pdf", ipynb_address])

[NbConvertApp] Converting notebook /content/drive/Shareddrives/Deep Learning Final Project/fake_reply_generation.ipynb to HTML
[NbConvertApp] Writing 596523 bytes to /content/drive/Shareddrives/Deep Learning Final Project/fake_reply_generation.html
[NbConvertApp] Converting notebook /content/drive/Shareddrives/Deep Learning Final Project/fake_reply_generation.ipynb to PDF
[NbConvertApp] Writing 243707 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: [u'xelatex', u'./notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: [u'bibtex', u'./notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 213410 bytes to /content/drive/Shareddrives/Deep Learning Final Project/fake_reply_generation.pdf


0